In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import time
import torch
from torch_geometric.loader import DataLoader
import argparse
import yaml
from torch_geometric.nn import radius
from torch.optim import Adam
import torch.autograd.profiler as profiler
from torch.profiler import profile, record_function, ProfilerActivity

from rbfConv import RbfConv
# from dataset import compressedFluidDataset, prepareData

import inspect
import re
def debugPrint(x):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} [{}] = {}".format(r,type(x).__name__, x))
# %matplotlib notebook
import copy

import time
import torch
from torch_geometric.loader import DataLoader
from tqdm.notebook import trange, tqdm
import argparse
import yaml
from torch_geometric.nn import radius
from torch.optim import Adam
import torch.autograd.profiler as profiler
from torch.profiler import profile, record_function, ProfilerActivity

from rbfConv import RbfConv
from dataset import compressedFluidDataset, prepareData

import inspect
import re
def debugPrint(x):
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} [{}] = {}".format(r,type(x).__name__, x))


import tomli
from scipy.optimize import minimize
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt

seed = 0


import random 
import numpy as np
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
# print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('running on: ', device)
torch.set_num_threads(1)

from joblib import Parallel, delayed

from cutlass import *
from rbfConv import *
from tqdm.notebook import tqdm

from datautils import *
# from sphUtils import *
from lossFunctions import *

from plotting import *
plt.style.use('dark_background')
from tqdm.notebook import trange, tqdm

In [2]:

parser = argparse.ArgumentParser()
parser.add_argument('-e','--epochs', type=int, default=25)
parser.add_argument('-cmap','--coordinateMapping', type=str, default='preserving')
parser.add_argument('-w','--windowFunction', type=str, default='poly6')
parser.add_argument('-c','--cutoff', type=int, default=1800)
parser.add_argument('-b','--batch_size', type=int, default=2)
parser.add_argument('--cutlassBatchSize', type=int, default=512)
parser.add_argument('-r','--lr', type=float, default=0.01)
parser.add_argument('--lr_decay_factor', type=float, default=0.9)
parser.add_argument('--lr_decay_step_size', type=int, default=1)
parser.add_argument('--weight_decay', type=float, default=0)
parser.add_argument('-x','--rbf_x', type=str, default='linear')
parser.add_argument('-y','--rbf_y', type=str, default='linear')
parser.add_argument('-n','--n', type=int, default=4)
parser.add_argument('-m','--m', type=int, default=4)
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--networkseed', type=int, default=42)
parser.add_argument('-d','--frameDistance', type=int, default=1)
parser.add_argument('--dataDistance', type=int, default=1)
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--gpus', type=int, default=1)
parser.add_argument('-f','--forwardLoss', type=bool, default=False)
parser.add_argument('-v','--verbose', type=bool, default=False)
parser.add_argument('-l','--li', type=bool, default=True)
parser.add_argument('-a','--activation', type=str, default='relu')
parser.add_argument('--arch', type=str, default='32 64 64 3')
parser.add_argument('--limitData', type=int, default=-1)
parser.add_argument('--iterations', type=int, default=1000)
parser.add_argument('-u', '--maxUnroll', type=int, default=10)
parser.add_argument('--minUnroll', type=int, default=2)
parser.add_argument('-augj', '--augmentJitter', type=bool, default=True)
parser.add_argument('-j', '--jitterAmount', type=float, default=0.01)
parser.add_argument('-augr', '--augmentAngle', type=bool, default=True)
parser.add_argument('-adjust', '--adjustForFrameDistance', type = bool, default = True)
parser.add_argument('-netArch', '--network', type=str, default='default')
parser.add_argument('-norm', '--normalized', type=bool, default=False)

_StoreAction(option_strings=['-norm', '--normalized'], dest='normalized', nargs=None, const=None, default=False, type=<class 'bool'>, choices=None, required=False, help=None, metavar=None)

In [6]:
import shlex

In [15]:
a = "--verbose TRUE --frameDistance 16 -n 4 -m 4 --arch '32 64 64 3' -x 'rbf cubic_spline' -y 'rbf cubic_spline' -w None -cmap cartesian --epochs 25 --networkseed 3 -netArch default"
a = shlex.split(a)
args = parser.parse_args(a)

In [28]:

if args.verbose:
    print('Setting all rng seeds to %d' % args.seed)
import random 
import numpy as np

random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)

if args.verbose:
    print('Available cuda devices:', torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if args.verbose:
    print('Running on Device %s' % device)
torch.set_num_threads(1)

# from joblib import Parallel, delayed

from cutlass import *
from rbfConv import *
from datautils import *
from plotting import *

# Use dark theme
plt.style.use('dark_background')
from tqdm import tqdm
import os

from rbfNet import *
from tqdm.notebook import trange, tqdm


if args.verbose:
    print('Parsing data in ../export')
basePath = '../export'
basePath = os.path.expanduser('~/dev/datasets/generative2D')

Setting all rng seeds to 42
Available cuda devices: 0
Running on Device cpu
Parsing data in ../export


In [17]:


# basePath = '~/dev/datasets/WBCSPH2Dc/train'
# basePath = os.path.expanduser(basePath)

# simulationFiles = [basePath + '/' + f for f in os.listdir(basePath) if f.endswith('.zst')]
trainingFiles = [basePath + '/train/' + f for f in os.listdir(basePath + '/train/') if f.endswith('.hdf5')]
# validationFiles = [basePath + '/valid/' + f for f in os.listdir(basePath + '/valid/') if f.endswith('.hdf5')]
# for i, c in enumerate(simulationFiles):
#     print(i ,c)
#     
# simulationFiles  = [simulationFiles[0]]
# simulationFiles = simulationFiles[:1]

training = []
validation = []
testing = []

    
    # for s in simulationFiles:    
#     _, train, valid, test = splitFileZSTD(s, split = True, limitRollOut = False, skip = 0, cutoff = 1800, distance = 1)
#     training.append((s,train))
#     validation.append((s,valid))
#     testing.append((s,test))
# debugPrint(training)

# simulationFiles = [basePath + '/' + f for f in os.listdir(basePath) if f.endswith('.hdf5')]

if args.limitData > 0:
    files = []
    for i in range(max(len(trainingFiles), args.limitData)):
        files.append(trainingFiles[i])
    simulationFiles = files
# simulationFiles = [simulationFiles[0]]
if args.verbose:
    print('Input files:')
    for i, c in enumerate(trainingFiles):
        print('\t', i ,c)


Input files:
	 0 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-49-55.hdf5
	 1 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-59-11.hdf5
	 2 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-08-03.hdf5
	 3 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-26-38.hdf5
	 4 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_17-13-28.hdf5
	 5 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-35-51.hdf5
	 6 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_17-03-59.hdf5
	 7 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-17-15.hdf5
	 8 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-31-08.hdf5
	 9 /home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-21-57.hdf5
	 10 /home/winchenbach/dev/datasets/g

In [18]:

training = []
validation = []
testing = []

for s in trainingFiles:
    f, s, u = splitFile(s, split = False, cutoff = -args.frameDistance * args.maxUnroll, skip = args.frameDistance if args.adjustForFrameDistance else 0)
    training.append((f, (s,u)))
# for s in tqdm(validationFiles):
#     f, s, u = splitFile(s, split = False, cutoff = -4, skip = 0)
#     validation.append((f, (s,u)))
    
if args.verbose:
    print('Processed data into datasets:')
    debugPrint(training)
    debugPrint(validation)
    debugPrint(testing)

Processed data into datasets:
training [list] = [('/home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-49-55.hdf5', (array([  16,   17,   18, ..., 3034, 3035, 3036]), array([8, 8, 8, ..., 3, 2, 1], dtype=int32))), ('/home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-59-11.hdf5', (array([  16,   17,   18, ..., 3034, 3035, 3036]), array([8, 8, 8, ..., 3, 2, 1], dtype=int32))), ('/home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-08-03.hdf5', (array([  16,   17,   18, ..., 3034, 3035, 3036]), array([8, 8, 8, ..., 3, 2, 1], dtype=int32))), ('/home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_16-26-38.hdf5', (array([  16,   17,   18, ..., 3034, 3035, 3036]), array([8, 8, 8, ..., 3, 2, 1], dtype=int32))), ('/home/winchenbach/dev/datasets/generative2D/train/generative_16 - 2023-03-10_17-13-28.hdf5', (array([  16,   17,   18, ..., 3034, 3035, 3036]), array([8, 8, 8, ..., 3, 2, 1], dt

In [19]:

batch_size = args.batch_size

if args.verbose:
    print('Setting up data loaders')
train_ds = datasetLoader(training)
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size = batch_size).batch_sampler

# validation_ds = datasetLoader(validation)
# validation_dataloader = DataLoader(validation_ds, shuffle=True, batch_size = batch_size).batch_sampler


Setting up data loaders


In [20]:

if args.verbose:
    print('Setting up network parameters:')
fileName, frameIndex, maxRollout = train_ds[len(train_ds)//2]
attributes, inputData, groundTruthData = loadFrame(fileName, frameIndex, 1 + np.arange(1))

fluidPositions, boundaryPositions, fluidFeatures, boundaryFeatures = constructFluidFeatures(attributes, inputData)

n = args.n
m = args.m
coordinateMapping = args.coordinateMapping
windowFn = getWindowFunction(args.windowFunction)
rbf_x = args.rbf_x
rbf_y = args.rbf_y
initialLR = args.lr
maxRollOut = 10
epochs = args.epochs
frameDistance = args.frameDistance

if args.verbose:
    print('Network Hyperparameters:')
    print('[n x m]: [%dx%d]'% (n, m))
    print('[rbf_x x rbf_y]: [%sx%s]'% (rbf_x, rbf_y))
    print('Mapping:', args.coordinateMapping)
    print('window function:', args.windowFunction)
    print('activation function:', args.activation)
    print('initial learning rate: ', initialLR)
    print('Training for %d epochs' % epochs)
    print('Rollout limit (if applicable):', maxRollout)
    print('Training with frame offset of', frameDistance)
    print('Network architecture', args.arch)



Setting up network parameters:
Network Hyperparameters:
[n x m]: [4x4]
[rbf_x x rbf_y]: [rbf cubic_splinexrbf cubic_spline]
Mapping: cartesian
window function: None
activation function: relu
initial learning rate:  0.01
Training for 25 epochs
Rollout limit (if applicable): 8
Training with frame offset of 16
Network architecture 32 64 64 3


In [22]:

widths = args.arch.strip().split(' ')
layers = [int(s) for s in widths]
# debugPrint(layers)
if args.verbose:
    print('Building Network')

random.seed(args.networkseed)
torch.manual_seed(args.networkseed)
torch.cuda.manual_seed(args.networkseed)
np.random.seed(args.networkseed)



model = None
if args.network == 'default':
    model = RbfNet(fluidFeatures.shape[1], boundaryFeatures.shape[1], layers = layers, coordinateMapping = coordinateMapping, n = n, m = m, windowFn = windowFn, rbf_x = rbf_x, rbf_y = rbf_y, batchSize = args.cutlassBatchSize, normalized = args.normalized)
if args.network == 'split':
    model = RbfSplitNet(fluidFeatures.shape[1], boundaryFeatures.shape[1], layers = layers, coordinateMapping = coordinateMapping, n = n, m = m, windowFn = windowFn, rbf_x = rbf_x, rbf_y = rbf_y, batchSize = args.cutlassBatchSize, normalized = args.normalized)
if args.network == 'interleaved':
    model = RbfInterleaveNet(fluidFeatures.shape[1], boundaryFeatures.shape[1], layers = layers, coordinateMapping = coordinateMapping, n = n, m = m, windowFn = windowFn, rbf_x = rbf_x, rbf_y = rbf_y, batchSize = args.cutlassBatchSize, normalized = args.normalized)
if args.network == 'input':
    model = RbfInputNet(fluidFeatures.shape[1], boundaryFeatures.shape[1], layers = layers, coordinateMapping = coordinateMapping, n = n, m = m, windowFn = windowFn, rbf_x = rbf_x, rbf_y = rbf_y, batchSize = args.cutlassBatchSize, normalized = args.normalized)
if args.network == 'output':
    model = RbfOutputNet(fluidFeatures.shape[1], boundaryFeatures.shape[1], layers = layers, coordinateMapping = coordinateMapping, n = n, m = m, windowFn = windowFn, rbf_x = rbf_x, rbf_y = rbf_y, batchSize = args.cutlassBatchSize, normalized = args.normalized)


lr = initialLR
optimizer = Adam(model.parameters(), lr=lr, weight_decay=args.weight_decay)
model = model.to(device)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
if args.verbose:
    print('Number of parameters', count_parameters(model))
    
    
optimizer.zero_grad()
model.train()

hyperParameterDict = {}
hyperParameterDict['n'] = n
hyperParameterDict['m'] = m
hyperParameterDict['coordinateMapping'] = coordinateMapping
hyperParameterDict['rbf_x'] = rbf_x
hyperParameterDict['rbf_y'] = rbf_y
hyperParameterDict['windowFunction'] =  args.windowFunction
hyperParameterDict['liLoss'] = 'yes' if args.li else 'no'
hyperParameterDict['initialLR'] = initialLR
hyperParameterDict['maxRollOut'] = maxRollOut
hyperParameterDict['epochs'] = epochs
hyperParameterDict['frameDistance'] = frameDistance
hyperParameterDict['dataDistance'] = args.dataDistance
hyperParameterDict['parameters'] =  count_parameters(model)
hyperParameterDict['cutoff'] =  args.cutoff
hyperParameterDict['dataLimit'] =  args.limitData 
hyperParameterDict['arch'] =  args.arch
hyperParameterDict['seed'] =  args.seed
hyperParameterDict['minUnroll'] =  args.minUnroll
hyperParameterDict['maxUnroll'] =  args.maxUnroll
hyperParameterDict['augmentAngle'] =  args.augmentAngle
hyperParameterDict['augmentJitter'] =  args.augmentJitter
hyperParameterDict['jitterAmount'] =  args.jitterAmount
hyperParameterDict['networkSeed'] =  args.networkseed
hyperParameterDict['network'] = args.network
hyperParameterDict['normalized'] = args.normalized
hyperParameterDict['adjustForFrameDistance'] = args.adjustForFrameDistance
lr = initialLR

Building Network
Number of parameters 181414


In [23]:

timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
networkPrefix = args.network

exportString = '%s - n=[%2d,%2d] rbf=[%s,%s] map = %s window = %s d = %2d e = %2d arch %s distance = %2d - %s seed %s' % (networkPrefix, hyperParameterDict['n'], hyperParameterDict['m'], hyperParameterDict['rbf_x'], hyperParameterDict['rbf_y'], hyperParameterDict['coordinateMapping'], args.windowFunction, hyperParameterDict['frameDistance'], hyperParameterDict['epochs'], args.arch, frameDistance, timestamp, args.networkseed)

shortLabel = '%14s [%14s] - %s -> [%16s, %16s] x [%2d, %2d] @ %2s ' % (hyperParameterDict['windowFunction'], hyperParameterDict['arch'], hyperParameterDict['coordinateMapping'], hyperParameterDict['rbf_x'], hyperParameterDict['rbf_y'], hyperParameterDict['n'], hyperParameterDict['m'],hyperParameterDict['networkSeed'])
# print(shortLabel)

# exit()
# if args.gpus == 1:

#     debugPrint(hyperParameterDict)
# if args.gpus == 1:
#     debugPrint(exportString)
if args.verbose:
    print('Writing output to ./trainingData/%s' % exportString)

# exportPath = './trainingData/%s - %s.hdf5' %(self.config['export']['prefix'], timestamp)
if not os.path.exists('./trainingData/%s' % exportString):
    os.makedirs('./trainingData/%s' % exportString)
# self.outFile = h5py.File(self.exportPath,'w')

Writing output to ./trainingData/default - n=[ 4, 4] rbf=[rbf cubic_spline,rbf cubic_spline] map = cartesian window = None d = 16 e = 25 arch 32 64 64 3 distance = 16 - 2023-06-06_20-58-13 seed 3


In [ ]:

def processBatch(model, device, li, e, unroll, train_ds, bdata, frameDistance, augmentAngle = False, augmentJitter = False, jitterAmount = 0.01, adjustForFrameDistance = True):
    with record_function("process batch"): 
        fluidPositions, boundaryPositions, fluidFeatures, boundaryFeatures, fluidGravity, fluidBatches, boundaryBatches, groundTruths, attributes = \
            loadBatch(train_ds, bdata, constructFluidFeatures, unroll, frameDistance, augmentAngle = augmentAngle, augmentJitter = augmentJitter, jitterAmount = jitterAmount, adjustForFrameDistance = adjustForFrameDistance)    


        predictedPositions = fluidPositions.to(device)
        predictedVelocity = fluidFeatures[:,1:3].to(device)

        bLosses = []
        boundaryPositions = boundaryPositions.to(device)
        fluidFeatures = fluidFeatures.to(device)
        boundaryFeatures = boundaryFeatures.to(device)
        fluidBatches = fluidBatches.to(device)
        boundaryBatches = boundaryBatches.to(device)

        gravity = torch.zeros_like(predictedVelocity)
        gravity = fluidGravity[:,:2].to(device)
        
    #     gravity[:,1] = -9.81

        for u in range(unroll):
            with record_function("prcess batch[unroll]"): 
    #         loss, predictedPositions, predictedVelocity = runNetwork(fluidPositions.to(device), inputData['fluidVelocity'].to(device), attributes['dt'], frameDistance, gravity, fluidFeatures, boundaryPositions.to(device), boundaryFeatures.to(device), groundTruths[0], model, None, None, True)
                loss, predictedPositions, predictedVelocity = runNetwork(predictedPositions, predictedVelocity, attributes[0], frameDistance, gravity, fluidFeatures, boundaryPositions, boundaryFeatures, groundTruths[u], model, fluidBatches, boundaryBatches, li)

                batchedLoss = []
                for i in range(len(bdata)):
                    L = loss[fluidBatches == i]
                    Lterms = (torch.mean(L), torch.max(torch.abs(L)), torch.min(torch.abs(L)), torch.std(L))            
                    batchedLoss.append(torch.hstack(Lterms))
                batchedLoss = torch.vstack(batchedLoss).unsqueeze(0)
                bLosses.append(batchedLoss)

        bLosses = torch.vstack(bLosses)
        maxLosses = torch.max(bLosses[:,:,1], dim = 0)[0]
        minLosses = torch.min(bLosses[:,:,2], dim = 0)[0]
        meanLosses = torch.mean(bLosses[:,:,0], dim = 0)
        stdLosses = torch.mean(bLosses[:,:,3], dim = 0)


        del predictedPositions, predictedVelocity, boundaryPositions, fluidFeatures, boundaryFeatures, fluidBatches, boundaryBatches

        bLosses = bLosses.transpose(0,1)

        return bLosses, meanLosses, minLosses, maxLosses, stdLosses




In [26]:

def processDataLoaderIter(iterations, e, rollout, ds, dataLoader, dataIter, model, optimizer, train = True, prefix = '', augmentAngle = False, augmentJitter = False, jitterAmount = 0.01):
    with record_function("prcess data loader"): 
        losses = []
        batchIndices = []

        if train:
            model.train(True)
        else:
            model.train(False)

        i = 0
        for b in (pbl := tqdm(range(iterations), leave=False)):
            try:
                bdata = next(dataIter)
            except:
                dataIter = iter(dataLoader)
                bdata = next(dataIter)
                
            with record_function("prcess data loader[batch]"): 
                if train:
                    optimizer.zero_grad()
                batchLosses, meanLosses, minLosses, maxLosses, stdLosses = processBatch(model, device, True, e, rollout, ds, bdata, frameDistance, augmentAngle, augmentJitter, jitterAmount, adjustForFrameDistance = args.adjustForFrameDistance)
                # print(torch.max(model.ni))
                
                batchIndices.append(np.array(bdata))
                losses.append(batchLosses.detach().cpu().numpy())

                with record_function("prcess data loader[batch] - backward"): 
                    sumLosses = torch.mean(batchLosses[:,:,0]) #+ torch.mean(batchLosses[:,:,1])
                    if train:
                        sumLosses.backward()
                        optimizer.step()
                lossString = np.array2string(torch.mean(batchLosses[:,:,0],dim=0).detach().cpu().numpy(), formatter={'float_kind':lambda x: "%.2e" % x})
                batchString = str(np.array2string(np.array(bdata), formatter={'float_kind':lambda x: "%.2f" % x, 'int':lambda x:'%04d' % x}))

#                 with portalocker.Lock('README.md', flags = 0x2, timeout = None):
                pbl.set_description('%8s[gpu %d]: %3d [%1d] @ %1.1e: :  %s -> %.2e' %(prefix, args.gpu, e, rollout, lr, batchString, sumLosses.detach().cpu().numpy()))
                pbl.update()
                if prefix == 'training':
                    # pb.set_description('[gpu %d] Learning: %1.4e Validation: %1.4e' %(args.gpu, np.mean(np.mean(np.vstack(losses)[:,:,0], axis = 1)), 0))
                    pb.set_description('[gpu %d] %90s - Learning: %1.4e' %(args.gpu, shortLabel, np.mean(np.mean(np.vstack(losses)[:,:,0], axis = 1))))
                if prefix == 'validation':
                    pb.set_description('[gpu %d] Learning: %1.4e Validation: %1.4e' %(args.gpu, trainLoss, np.mean(np.mean(np.vstack(losses)[:,:,0], axis = 1))))
                pb.update()
#                 i = i + 1
#                 if i > 100:
#                     break
        bIndices  = np.hstack(batchIndices)
        losses = np.vstack(losses)

        # idx = np.argsort(bIndices)
        # bIndices = bIndices[idx]
        # losses = losses[idx]

        epochLoss = losses
        return epochLoss

In [31]:
training = {}
validation = {}
testing = {}

trainLoss = 0
validationLoss = 0
train_iter = iter(train_dataloader)

trainingEpochLosses = []
trainingEpochLosses2 = []
validationLosses = []

unroll = 2

random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
np.random.seed(args.seed)
# if args.verbose:
    # print('Start of training')

pb = tqdm(range(epochs * args.iterations))
# pb.reset(total=len(train_dataloader))
for epoch in range(epochs):
    losses = []

    unroll = max(args.minUnroll, min(epoch // 2 + 1, args.maxUnroll))
    # trainingEpochLoss = processDataLoaderIter(args.iterations, epoch, epoch // 2 + 1, train_ds, train_dataloader, train_iter, model, optimizer, True, prefix = 'training', augmentAngle=args.argumentAngle, augmentJitter=args.augmentJitter, jitterAmount=args.jitterAmount)
    trainingEpochLoss = processDataLoaderIter(args.iterations, epoch, unroll, train_ds, train_dataloader, train_iter, model, optimizer, True, prefix = 'training', augmentAngle=args.augmentAngle, augmentJitter=args.augmentJitter, jitterAmount=args.jitterAmount)

#     trainingEpochLoss = processDataLoader(epoch,unroll, train_ds, train_dataloader, model, optimizer, True, prefix = 'training')
    trainingEpochLosses.append(trainingEpochLoss)
    # torch.save(model.state_dict(), './trainingData/%s/model_%03d.torch' % (exportString, epoch))
    if epoch % 5 == 0 and epoch > 0:
        lr = lr * 0.5
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']
    torch.save(model.state_dict(), './trainingData/%s/model_%03d.torch' % (exportString, epoch))


  0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 